# クイックスタート2

このノートのクイックスタートは、2025年2月時点での Microsoft の Azure AI Foundry の[ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-code)に記載されている内容をベースに、実践的な内容に修正したものです。

※仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。


## 前提条件


このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Azure AI Foundry プロジェクトに接続済みの Application Insights インスタンス（トレースに利用） 
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects azure-ai-inference opentelemetry-sdk azure-core-tracing-opentelemetry

## Azure AI Foundry で シンプルな AI 推論チャットアプリを開発する (続)


このノートでは、Azure AI Foundry SDK を使用して、シンプルチャットアプリケーションを開発する方法について説明します。
Python アプリケーションのコードの一部として定義したプロンプトで 推論モデル（LLM）を呼び出して、推論モデル（LLM）からの出力を評価します。

このクイックスタートでは、クイックスタート1 から以下のポイントを追加したサンプルです。

- `PromptTemplate` を利用して、定型のテンプレートを準備する
- AI 推論 API のトレースを有効にする


### 準備


このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o-mini` モデルをプロジェクトへデプロイしてください。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。


### チャットアプリのコードサンプル

#### 1. AI プロジェクトに接続します


チャット補完クライアントを使って問い合わせを行うための準備を行います。
クイックスタート 1 で使ったコードを使います。

  - `.env` ファイルの内容を環境変数に読み込みます。
    - プロジェクトの接続文字列：PROJECT_CONNECTION_STRING="**<AI Foundry プロジェクトの接続文字列>**"
    - 推論モデルの名前：MODEL_NAME="**<デプロイしたLLMモデル名>**"
  - 現在のユーザーで認証するための Azure 資格情報を取得します。
  - `AIProjectClient` の `from_connection_string()` を使って、Azure AI Foundry プロジェクトに接続します。


In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from opentelemetry import trace
from azure.core.settings import settings

# Enable Azure SDK tracing with either of two lines:
# os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"
# settings.tracing_implementation = "opentelemetry"
tracer = trace.get_tracer(__name__)

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')

try:
    credential = DefaultAzureCredential()
    print(f"✅ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"❌ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print(f"✅ プロジェクトクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ プロジェクトクライアントの初期化が以下の理由で失敗しました: {str(e)}")

プロジェクトに接続された AI サービス（チャットク補完ライアント）を初期化します。
`get_chat_completions_client()` を使って、チャット補完クライアント `chat` (`ChatCompletionsClient` クラスのインスタンス) を作成します。


In [ ]:
try:
    chat = project.inference.get_chat_completions_client()
    print(f"✅ チャットクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ チャットクライアントの初期化が以下の理由で失敗しました: {str(e)}")

#### 2. トレースを有効にします


以下のコードでは、チャットアプリケーションのトレースを有効にします。
トレースを有効にするために必要な Python ライブラリを追加します。

> AI プロジェクトに Application Insights インスタンスを接続していない場合はこのセクションをスキップしてください。

- `AIInferenceInstrumentor` を使って、Azure AI 推論 SDK のインストルメンテーションを有効にします。
- `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED` 環境変数を `true` に設定して、プロンプトと応答の内容をキャプチャします。
- Azure SDK トレースを有効にするには、`AZURE_SDK_TRACING_IMPLEMENTATION` 環境変数を `opentelemetry` に設定します。
  または、以下のコードを追加します。

    ```python
    from azure.core.settings import settings
    settings.tracing_implementation = "opentelemetry"
    ```
    
- トレース情報を Azure Application Insights (Azure Monitor) に送信するには、`configure_azure_monitor()` で有効にします。
    - ここでは Python 向けの `azure.monitor.opentelemetry` ライブラリを利用して、直接、送信（エクスポート）します。
    - AI プロジェクトに接続された Azure Application Insights への接続文字列を取得して、`configure_azure_monitor()`に渡します。


In [ ]:
%pip install azure-monitor-opentelemetry

In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable tracing
AIInferenceInstrumentor().instrument()
# Enable logging AI message contents
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

application_insights_connection_string = project.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("❌ Application Insights がこのプロジェクトで有効になっていません.")
    print("==> Azure AI Foundry ポータルの \"トレース\"タブから、Application Inights を有効にしてください.")
else:
    configure_azure_monitor(connection_string=application_insights_connection_string)
    print("✅ Application Insights によるトレースを有効にしました.")

# project.telemetry.enable(destination=sys.stdout)

#### 3. プロンプト テンプレートを作成します


推論モデルに問い合わせるときに利用するシステムプロンプトを、プロンプトテンプレート文字列として定義します。プロンプトテンプレートには外部から挿入する可変の引数を含めることができ、動的なプロンプトを構築できます。このプログラムでは、プロンプトテンプレートを作成して、チャットクライアントに渡します。

ここではサンプルとして、2種類の異なるシステムプロンプトのテンプレートを用意しています。

`{{first_name}}` と `{{last_name}}` がプロンプトテンプレートの可変の引数として定義されています。

In [ ]:
prompt_template_string="""
  system:
  あなたは 2350 年のテクノ パンク ロッカーのように話す AI アシスタントです。クールに、でもクール過ぎないようにしましょう。わかりましたか??
  ユーザーをファーストネームで呼び、ラストネームをダジャレにしてみてください。

  ユーザーのファーストネームは {{first_name}}、ラストネームは {{last_name}} です
"""

In [ ]:
prompt_template_string="""
  system:
  あなたはグルメと旅行に詳しいコンシェルジュです。質問に対して現在の流行とその詳細な情報を付け加えて説明してください。わかりましたか??
  ユーザーをラストネームに「様」を付けて呼んでください。

  ユーザーのファーストネームは {{first_name}}、ラストネームは {{last_name}} です。
"""

#### 4. 推論モデルからの回答を取得する関数を定義します


`PromptTemple` の `from_string()` を使ってプロンプトテンプレートを作成します。
プロンプトテンプレートの可変の引数のデータ (`context`) を受け取り、`create_message()` を使って、チャット補完クライアントに渡すシステムプロンプトを作成します。

システムプロンプトと問い合わせメッセージを組み合わせて、チャット補完クライアントを使って問い合わせを行い回答を取得 (`complete()`) します。


In [ ]:
from azure.ai.inference.prompts import PromptTemplate

@tracer.start_as_current_span("quick_start_02_get_chat_response")
def get_chat_response(messages, context):
    # create a prompt template from an inline string (using mustache syntax)
    prompt_template = PromptTemplate.from_string(prompt_template_string)

    # generate system message from the template, passing in the context as variables
    system_message = prompt_template.create_messages(data=context)

    # add the prompt messages to the user messages
    return chat.complete(
        model=model_name_string,
        messages=system_message + messages,
        temperature=1,
        frequency_penalty=0.5,
        presence_penalty=0.5,
    )

#### 5. 推論モデルへの質問を作成します

ユーザーのクエリとユーザーの名前のデータを定義します。
ここでは2つのサンプルデータを用意しましたので、どちらかを使ってください。

In [ ]:
query = "世界で一番おいしい食べ物がある都市はどこですか?"
context={"first_name": "一郎", "last_name": "佐藤"}

In [ ]:
query = "世界で一番綺麗な場所がある都市はどこですか?"
context={"first_name": "五郎", "last_name": "田中"}

#### 6. アプリケーションを実行します


`__main__` 関数を呼び出し、アプリケーションを実行します。


In [ ]:
from azure.ai.inference.models import UserMessage

if __name__ == "__main__":
    response = get_chat_response(
        messages=[UserMessage(content=query)],
        context=context,
    )
    print(response.choices[0].message.content)

チャット補完クライアントからのレスポンスの内容 (JSON) を表示します。

In [ ]:
print(response)